In [2]:
import pandas as pd
import numpy as np
import sqlite3

engine = sqlite3.connect('yahoo')

In [99]:
click_cols = ['time', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6', 'displayed', 'clicked']

In [115]:
articles_df = pd.read_sql_query('SELECT * FROM articles',con=engine).set_index('index')
clicks_df = pd.read_sql_query('''SELECT * FROM clicks WHERE (ABS(CAST((RANDOM()) as int)) % 100) < 5''', con=engine)
clicks_df[click_cols] = clicks_df[click_cols].apply(pd.to_numeric, ) # Temporary workaround, this should be done before sql

In [130]:
#articles_df.dtypes

In [117]:
articles_df.head()

,2,3,4,5,6,1
index,,,,,,
109498,0.306008,0.000450,0.077048,0.230439,0.386055,1.0
109509,0.306008,0.000450,0.077048,0.230439,0.386055,1.0
109508,0.264355,0.000012,0.037393,0.420649,0.277591,1.0
109473,0.295442,0.000014,0.135191,0.292304,0.277050,1.0
109503,0.306008,0.000450,0.077048,0.230439,0.386055,1.0


In [118]:
clicks_df.shape

(234140, 11)

In [121]:
#clicks_df.dtypes

In [120]:
 clicks_df.head()

,index,time,user_1,user_2,user_3,user_4,user_5,user_6,article_pool,displayed,clicked
0,24,1241160900,0.008325,0.079014,0.001413,0.911076,0.000172,1.0,"['109498', '109509', '109508', '109473', '1095...",109510,0
1,59,1241160900,0.003677,0.000000,0.986935,0.008073,0.001315,1.0,"['109498', '109509', '109508', '109473', '1095...",109503,0
2,65,1241160900,0.042438,0.150965,0.005879,0.800057,0.000661,1.0,"['109498', '109509', '109508', '109473', '1095...",109495,0
3,85,1241160900,0.553029,0.045877,0.094129,0.291508,0.015457,1.0,"['109498', '109509', '109508', '109473', '1095...",109498,0
4,114,1241160900,0.553029,0.045877,0.094129,0.291508,0.015457,1.0,"['109498', '109509', '109508', '109473', '1095...",109506,0


In [1]:
"""
Evaluation Protocol
===================
Let σ be an input stream of logged events to be used for the simulation,
where an event consists of a tuple consisting of the user vector, 
20 article vectors (context vectors), the displayed vector (selected arm), 
and the result (observed reward).
Step through each element s sampled from the stream.
Let t denote the current time-step in the algorithm and h_{t-1} prior history.
If given h_{t-1} and the current context, the algorithm picks the same vector
as the selected arm - retain the event and add it to the history. Otherwise,
keep processing items.
"""

In [177]:
""" Playing around with how to convert Yahoo data to good format for evaluation. """

from numpy import transpose
from numpy.linalg import pinv, det

np.set_printoptions(precision=3)

user_feat_mat = clicks_df.as_matrix(['user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6'])
X = np.empty((1, 20, 12))
init_iter = 2
for i in range(init_iter):
    # Extract all article vectors for first log item.
    article_vecs = np.array([articles_df.ix[x].values for x in eval(clicks_df.ix[i]['article_pool'])])
    # For each of the article row vectors, add a copy of the user features.
    tmp = np.append(article_vecs, values=np.tile(user_feat_mat[i], (len(article_vecs), 1)), axis=1)
    tmp = tmp.reshape((1, 20, 12)) # (timestep, arm, feature)
    X = np.append(X, tmp, axis=0)  # Feature vector of 6 article features and 6 user features
print(X.shape)

_Xti = X[:init_iter+1,0]
print(_Xti)
_XtiT = transpose(_Xti)
_XTX = pinv(_XtiT.dot(_Xti))
_Yti = Y[:t+1,i]

(3, 20, 12)
[[  6.933e-310   6.933e-310   1.076e-315   1.076e-315   4.941e-324
    1.076e-315   4.941e-324   1.076e-315   1.235e-322   6.933e-310
    9.881e-324   1.076e-315]
 [  3.060e-001   4.500e-004   7.705e-002   2.304e-001   3.861e-001
    1.000e+000   8.325e-003   7.901e-002   1.413e-003   9.111e-001
    1.720e-004   1.000e+000]
 [  3.060e-001   4.500e-004   7.705e-002   2.304e-001   3.861e-001
    1.000e+000   3.677e-003   0.000e+000   9.869e-001   8.073e-003
    1.315e-003   1.000e+000]]


In [ ]:
def top_interval(X, Y, k, d, _delta, T):
    """
    Simulates T rounds of TopInterval for k.
    
    :param X: a 3-axis (T, k, d) ndarray of d-dimensional context vectors for each
              time-step and arm
    :param Y: a T x k ndarray of reward function output for each context vector
    :param k: the number of arms
    :param d: the number of features
    :param _delta: confidence parameter
    :param T: the number of iterations
    """
    _eta = eta(T)                               # exploration cutoff probabilities
    picks = []
    for t in range(T):
        print('Iteration [{0} / {1}]'.format(t, T))